# 텍스트마이닝. 감성분석과 토픽분석

### - pandas 버전이 1.1.4 이상인지 확인하고, 아니라면 upgrade 하기. (pyLDA 에러 때문)

In [1]:
import pandas as pd
#pd.show_versions()

## pandas 버전이 1.1.4 이상이 아니라면, --upgrade 명령으로 버전을 업그레이드


### - Anaconda Prompt 를 [관리자권한으로 실행] 한 후에, 명령어 입력:  pip  install  --upgrade  pandas 
###    -> 업그레이드 설치를 적용하기 위해서, Jupyter Notobook을 종료했다가 다시 실행하기 !!

In [2]:
import pandas as pd
pd.__version__

'1.3.4'

In [ ]:
#!pip uninstall numpy
#!pip uninstall pandas

#!pip install --user numpy
#!pip install --user  pandas

### 한글 UnicoedEncodingError를 방지하기 위해 기본 인코딩을 "utf-8"로 설정

In [3]:
import os
os.environ["PYTHONIOENCODING"] = "utf-8"

### 경고메시지 표시 안하게 설정하기

In [4]:
import warnings
warnings.filterwarnings(action='ignore')

#  감성 분류 모델 구축

## 1. 데이터 수집

#### 깃허브에서 데이터 파일 다운로드 : https://github.com/e9t/nsmc 

## 2. 데이터 준비 및 탐색

### 2-1) 훈련용 데이터 준비

#### (1) 훈련용 데이터 파일 로드

In [5]:
nsmc_train_df = pd.read_csv('./sample_data/ratings_train.txt', encoding='utf8', sep='\t', engine='python')
nsmc_train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


#### (2) 데이터의 정보 확인

In [6]:
nsmc_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


#### (3) 'document'칼럼이 Null인 샘플 제거

In [7]:
nsmc_train_df = nsmc_train_df[nsmc_train_df['document'].notnull()]

In [8]:
nsmc_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


#### (4) 타겟 컬럼 label 확인 (0: 부정감성,   1: 긍정감성)

In [9]:
nsmc_train_df['label'].value_counts()

0    75170
1    74825
Name: label, dtype: int64

#### (5) 한글 이외의 문자는 공백으로 변환 (정규표현식 이용)

In [10]:
import re

In [11]:
nsmc_train_df['document'] = nsmc_train_df['document'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
nsmc_train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


### 2-2) 평가용 데이터 준비

#### (1) 평가용 데이터 파일 로드

In [12]:
nsmc_test_df = pd.read_csv('./sample_data/ratings_test.txt', encoding='utf8', sep='\t', engine='python')
nsmc_test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


#### (2) 데이터의 정보 확인

In [13]:
nsmc_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


#### (3) 'document'칼럼이 Null인 샘플 제거

In [14]:
nsmc_test_df = nsmc_test_df[nsmc_test_df['document'].notnull()]

#### (4) 타겟 컬럼 label 확인 (0: 부정감성, 1: 긍정감성)

In [15]:
print(nsmc_test_df['label'].value_counts())

1    25171
0    24826
Name: label, dtype: int64


#### (5) 한글 이외의 문자는 공백으로 변환 (정규표현식 이용)

In [16]:
nsmc_test_df['document'] = nsmc_test_df['document'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))

## 3. 분석 모델 구축

### 3-1) 피처 벡터화 : TF-IDF

#### (1) 형태소를 분석하여 토큰화 : 한글 형태소 엔진으로 Okt 이용

In [ ]:
#!pip install konlpy

In [17]:
from konlpy.tag import Okt

okt = Okt()

In [18]:
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

#### (2) TF-IDF 기반 피처 벡터 생성 : 실행시간 10분 정도 걸립니다 ☺

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer = okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf.fit(nsmc_train_df['document'])
nsmc_train_tfidf = tfidf.transform(nsmc_train_df['document'])

### 3-2) 감성 분류 모델 구축 : 로지스틱 회귀를 이용한 이진 분류

### - Sentiment Analysis using Logistic Regression

#### (1) 로지스틱 회귀 기반 분석모델 생성

In [20]:
from sklearn.linear_model import LogisticRegression

SA_lr = LogisticRegression(random_state = 0)

In [21]:
SA_lr.fit(nsmc_train_tfidf, nsmc_train_df['label'])

LogisticRegression(random_state=0)

#### (2) 로지스틱 회귀의  best 하이퍼파라미터 찾기

In [22]:
from sklearn.model_selection import GridSearchCV

params = {'C': [1, 3, 3.5, 4, 4.5, 5]}
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)

#### (3) 최적 분석 모델 훈련

In [23]:
SA_lr_grid_cv.fit(nsmc_train_tfidf, nsmc_train_df['label'])

Fitting 3 folds for each of 6 candidates, totalling 18 fits


GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 3.5, 4, 4.5, 5]}, scoring='accuracy',
             verbose=1)

In [24]:
print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_, 4))

{'C': 3} 0.8553


In [28]:
# 최적 파라미터의 best 모델 저장
SA_lr_best = SA_lr_grid_cv.best_estimator_

## 4. 분석 모델 평가

### 4-1) 평가용 데이터를 이용하여 감성 분석 모델 정확도

In [25]:
# 평가용 데이터의 피처 벡터화 : 실행시간 6분 정도 걸립니다 ☺
nsmc_test_tfidf = tfidf.transform(nsmc_test_df['document'])

In [29]:
test_predict = SA_lr_best.predict(nsmc_test_tfidf)

In [30]:
from sklearn.metrics import accuracy_score

print('감성 분석 정확도 : ', round(accuracy_score(nsmc_test_df['label'], test_predict), 3))

감성 분석 정확도 :  0.857


### 4-2) 새로운 텍스트에 대한 감성 예측

In [36]:
st = input('감성 분석할 문장입력 >> ')

감성 분석할 문장입력 >> 나는 지금 등교중입니다.


In [37]:
# 0) 입력 텍스트에 대한 전처리 수행
st = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(st)
print(st)
st = [" ".join(st)]
print(st)

['나는', '지금', '등교중입니다']
['나는 지금 등교중입니다']


In [38]:
# 1) 입력 텍스트의 피처 벡터화
st_tfidf = tfidf.transform(st)

# 2) 최적 감성분석 모델에 적용하여 감성 분석 평가
st_predict = SA_lr_best.predict(st_tfidf)

In [39]:
# 3) 예측 값 출력하기
if(st_predict== 0):
    print(st , "->> 부정 감성")
else :
    print(st , "->> 긍정 감성")

['나는 지금 등교중입니다'] ->> 긍정 감성
